In [1]:
import pandas as pd 
df = pd.read_csv('C:/Users/JunKai/Downloads/GiveMeSomeCredit/cs-training.csv')
print(df.head())

   Unnamed: 0  SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines  age  \
0           1                 1                              0.766127   45   
1           2                 0                              0.957151   40   
2           3                 0                              0.658180   38   
3           4                 0                              0.233810   30   
4           5                 0                              0.907239   49   

   NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
0                                     2   0.802982         9120.0   
1                                     0   0.121876         2600.0   
2                                     1   0.085113         3042.0   
3                                     0   0.036050         3300.0   
4                                     1   0.024926        63588.0   

   NumberOfOpenCreditLinesAndLoans  NumberOfTimes90DaysLate  \
0                               13                   

In [2]:
df['SeriousDlqin2yrs'].value_counts()#imbalanced classes

0    139974
1     10026
Name: SeriousDlqin2yrs, dtype: int64

In [3]:
df['NumberOfDependents'].value_counts()

0.0     86902
1.0     26316
2.0     19522
3.0      9483
4.0      2862
5.0       746
6.0       158
7.0        51
8.0        24
9.0         5
10.0        5
13.0        1
20.0        1
Name: NumberOfDependents, dtype: int64

In [4]:
df.isna().sum()

Unnamed: 0                                  0
SeriousDlqin2yrs                            0
RevolvingUtilizationOfUnsecuredLines        0
age                                         0
NumberOfTime30-59DaysPastDueNotWorse        0
DebtRatio                                   0
MonthlyIncome                           29731
NumberOfOpenCreditLinesAndLoans             0
NumberOfTimes90DaysLate                     0
NumberRealEstateLoansOrLines                0
NumberOfTime60-89DaysPastDueNotWorse        0
NumberOfDependents                       3924
dtype: int64

In [5]:
import numpy as np 
df['NumberOfDependents'].median()
df['NumberOfDependents'].replace(np.nan, df['NumberOfDependents'].median(), inplace=True)

In [6]:
df['MonthlyIncome'].median()
df['MonthlyIncome'].replace(np.nan, df['MonthlyIncome'].median(), inplace=True)

In [7]:
def f(x):
    if x == 0:
        return 'Zero_Dependents'
    else:
        return 'Morethanzero_Dependents'

In [8]:
df['Indicator_Dependents'] = df['NumberOfDependents'].apply(f)
df['Indicator_Dependents'] = pd.get_dummies(df['Indicator_Dependents'])


In [9]:
df['TotalNumberofTimes'] = df['NumberOfTime30-59DaysPastDueNotWorse']+df['NumberOfTime60-89DaysPastDueNotWorse']+df['NumberOfTimes90DaysLate']
df['TotalNumberofLoans'] = df['NumberRealEstateLoansOrLines']+df['NumberOfOpenCreditLinesAndLoans']

In [10]:
df = df.drop(['NumberOfTime30-59DaysPastDueNotWorse',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse'],axis = 1)


In [11]:
df.rename(columns = {'Unnamed: 0':'ID', 'RevolvingUtilizationOfUnsecuredLines':'CreditBalance', 
                              'SeriousDlqin2yrs':'RiskofDefaulting'}, inplace = True) 

In [12]:
X=df[['ID', 'CreditBalance', 'age', 'DebtRatio',
       'MonthlyIncome', 'NumberOfDependents', 'Indicator_Dependents', 'TotalNumberofTimes',
       'TotalNumberofLoans']]

In [13]:
y = df[['RiskofDefaulting']].values.ravel()

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.93


In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[41936    29]
 [ 3001    34]]


In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.97     41965
           1       0.54      0.01      0.02      3035

    accuracy                           0.93     45000
   macro avg       0.74      0.51      0.49     45000
weighted avg       0.91      0.93      0.90     45000



In [17]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 5)
rfe = rfe.fit(X, y)
print(rfe.support_)
print(rfe.ranking_)

C:\Users\JunKai\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=5 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
C:\Users\JunKai\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[False  True  True False False  True False  True  True]
[5 1 1 3 2 1 4 1 1]


In [18]:
X = df[['CreditBalance','age', 'NumberOfDependents', 'TotalNumberofTimes',
       'TotalNumberofLoans']]
y = df[['RiskofDefaulting']].values.ravel()

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.93


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.97     41965
           1       0.58      0.01      0.03      3035

    accuracy                           0.93     45000
   macro avg       0.76      0.51      0.50     45000
weighted avg       0.91      0.93      0.90     45000



In [21]:
X=df[['ID', 'CreditBalance', 'age', 'DebtRatio',
       'MonthlyIncome', 'NumberOfDependents', 'Indicator_Dependents', 'TotalNumberofTimes',
       'TotalNumberofLoans']]
y = df[['RiskofDefaulting']].values.ravel()

In [22]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

In [23]:
columns

Index(['ID', 'CreditBalance', 'age', 'DebtRatio', 'MonthlyIncome',
       'NumberOfDependents', 'Indicator_Dependents', 'TotalNumberofTimes',
       'TotalNumberofLoans'],
      dtype='object')

In [24]:
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['RiskofDefaulting'])

In [25]:
os_data_X['NumberOfDependents'] = os_data_X['NumberOfDependents'].replace(np.nan, os_data_X['NumberOfDependents'].median())

In [26]:
os_data_X.isna().sum()

ID                      0
CreditBalance           0
age                     0
DebtRatio               0
MonthlyIncome           0
NumberOfDependents      0
Indicator_Dependents    0
TotalNumberofTimes      0
TotalNumberofLoans      0
dtype: int64

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(os_data_X, os_data_y.values.ravel(), test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.72


C:\Users\JunKai\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[25562  3716]
 [12901 16627]]


In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.87      0.75     29278
           1       0.82      0.56      0.67     29528

    accuracy                           0.72     58806
   macro avg       0.74      0.72      0.71     58806
weighted avg       0.74      0.72      0.71     58806



In [30]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(os_data_X, os_data_y, test_size=0.3, random_state=0)

classifier = DecisionTreeClassifier(random_state = 0)  
  

classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)
print('Accuracy of decision tree classifier on test set: {:.2f}'.format(classifier.score(X_test, y_test)))

Accuracy of decision tree classifier on test set: 0.89


In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.87      0.88     29278
           1       0.88      0.90      0.89     29528

    accuracy                           0.89     58806
   macro avg       0.89      0.89      0.89     58806
weighted avg       0.89      0.89      0.89     58806



In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(os_data_X, os_data_y.values.ravel(), test_size=0.3, random_state=0)
from sklearn.ensemble import RandomForestClassifier


clf=RandomForestClassifier(n_estimators=100)


clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [33]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.923069074584226


In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92     29278
           1       0.93      0.92      0.92     29528

    accuracy                           0.92     58806
   macro avg       0.92      0.92      0.92     58806
weighted avg       0.92      0.92      0.92     58806



In [35]:
import pandas as pd
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

CreditBalance           0.238897
TotalNumberofTimes      0.172848
NumberOfDependents      0.158842
Indicator_Dependents    0.080961
ID                      0.079388
DebtRatio               0.078233
age                     0.076854
MonthlyIncome           0.070534
TotalNumberofLoans      0.043443
dtype: float64

In [36]:
X=df[['CreditBalance', 'NumberOfDependents','TotalNumberofTimes']]
y = df[['RiskofDefaulting']]
from sklearn.ensemble import RandomForestClassifier


clf=RandomForestClassifier(n_estimators=100)


clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [37]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9237152671496106


In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92     29278
           1       0.93      0.92      0.92     29528

    accuracy                           0.92     58806
   macro avg       0.92      0.92      0.92     58806
weighted avg       0.92      0.92      0.92     58806



In [39]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(os_data_X, os_data_y.values.ravel(), test_size=0.3, random_state=0)
clf = GradientBoostingClassifier(random_state=0)
clf.fit(X_train, y_train)
y_pred=clf.predict(X_test)


In [40]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8824609733700642


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.91      0.88     29278
           1       0.90      0.86      0.88     29528

    accuracy                           0.88     58806
   macro avg       0.88      0.88      0.88     58806
weighted avg       0.88      0.88      0.88     58806

